In [ ]:
## IMPORTS
import os
#Cameo dependencies
import cameo
from cameo.strain_design.deterministic.linear_programming import OptKnock
from cameo import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.flux_analysis.simulation import lmoma, pfba
#Data processing dependencies
import pandas as pd
import numpy as np
import ast
import plotly.express as px
from itertools import combinations
#Cobra dependencies
import cobra
from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model, save_matlab_model
from cobra.flux_analysis import production_envelope, flux_variability_analysis
from cobra.flux_analysis.variability import find_essential_genes
from cobra.sampling import sample
from utils.designFunctions import *
from cobra.sampling import OptGPSampler, ACHRSampler
import gurobipy

In [ ]:
gcfront_strategies_df = pd.read_csv('../data/gcfront_strategies_evaluation.csv')
configured_model = cobra.io.load_matlab_model('../models/configured_model.mat')
target_biomass = 'BiomassKT2440_ME'
target_reaction = 'DM_C80aPHA'
carbon_source = 'EX_pet(e)'

In [ ]:
result = phenotypic_phase_plane(configured_model,
                               variables=[configured_model.reactions.get_by_id(target_biomass)],
                               objective=configured_model.reactions.get_by_id(target_reaction),
                               points=20)

In [168]:
import plotly.graph_objects as go
r_df = result.data_frame

#parameter derived from dataframe
x = r_df.iloc[:, 0].tolist()
y1 = r_df.iloc[:, 1].tolist()
y2 = r_df.iloc[:, 2].tolist()

#user needed input:
title = "<b>Phenotypic Phase Plane with all design yields</b>"
xaxes_title = "<b>Growth (h<sup>-1</sup>)</b>"
yaxes_title =  "<b>PHA production (mMol/h)</b>"
strategy_yields = [(row['GrowthRate'], row['ProductFlux']) for index, row in gcfront_strategies_df.iterrows()]
color_list = gcfront_strategies_df.Color.tolist()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, 
    y=y1,
    fill=None,
    mode='lines',
    line_color='linen',
    showlegend=False
    ))
fig.add_trace(go.Scatter(
    x=x,
    y=y2,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', 
    line_color='linen',
    name='Design Space'))

fig.add_trace(go.Scatter(
    x=[a for a, b in strategy_yields], 
    y=[b for a, b in strategy_yields],
    mode='markers',
    marker = dict(color = color_list),
    showlegend=False
    ))

colorbar_range = gcfront_strategies_df.Score.tolist()

colorbar_trace  = go.Scatter(x=[None],
                             y=[None],
                             mode='markers',
                             marker=dict(
                                 colorscale=px.colors.sequential.Viridis, 
                                 showscale=True,
                                 cmin=min(colorbar_range),
                                 cmax=max(colorbar_range),
                                 colorbar=dict(title=dict(text='Design Score',font=dict(size=28)),
                                               thickness=16,
                                               len=0.5,
                                               tickvals=[min(colorbar_range), max(colorbar_range)],
                                               ticktext=[min(colorbar_range), max(colorbar_range)],
                                               tickfont=dict(size=18),
                                               outlinewidth=1,
                                               x=0.71,
                                               y=0.625)
                             ),
                             hoverinfo='none',
                             showlegend=False
                            )

fig.add_trace(colorbar_trace)

fig.update_layout(coloraxis_colorbar_x=-1)

fig.update_layout(
    width = 860,
    height = 800,
    title = dict(text=title, font=dict(size=38), x=0.5, xanchor='center')
)

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="right",
        x=0.99,
        font=dict(size=28)
))

fig.update_xaxes(title=dict(text=xaxes_title, font=dict(size=40)) ,
                 range=[0, 1.05*max(x)],
                 showline=True,
                 linewidth=3.0,
                 linecolor='black')

fig.update_yaxes(title=dict(text=yaxes_title, font=dict(size=40)),
                 range=[0, 1.05*max(y2)],
                 showline=True,
                 linewidth=3.0,
                 linecolor='black')

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis = dict(tickfont = dict(size=28)),
    xaxis = dict(tickfont = dict(size=28))
)
fig.write_image("envelope_with_designs.svg") #after editing this is figure 3A
fig.show()

In [ ]:
all_gdls = ['ABUTD','ALAR','ALATA_L','FORGLUIH2','ICDHyr','ICL','MALS','MCITL2','OARGDC','ORNCD','PDHbr','PDHcr','PPCSCT','RBK','SUCOAS','UPPN']
minimum_gdls = ['ORNCD','ICDHyr','ICL','MALS' ]
intermediate_gdls = minimum_gdls + ['FORGLUIH2','MCITL2','PPCSCT','SUCOAS']


gdls_kos_model = configured_model.copy()
for r in minimum_gdls:
    gdls_kos_model.reactions.get_by_id(r).bounds = (0,0)

min_result = phenotypic_phase_plane(gdls_kos_model,
                                    variables=[gdls_kos_model.reactions.get_by_id(target_biomass)],
                                    objective=gdls_kos_model.reactions.get_by_id(target_reaction),
                                    points=20)

gdls_kos_model = configured_model.copy()
for r in intermediate_gdls:
    gdls_kos_model.reactions.get_by_id(r).bounds = (0,0)


int_result = phenotypic_phase_plane(gdls_kos_model,
                                    variables=[gdls_kos_model.reactions.get_by_id(target_biomass)],
                                    objective=gdls_kos_model.reactions.get_by_id(target_reaction),
                                    points=20)


gdls_kos_model = configured_model.copy()
for r in gdls_result:
    gdls_kos_model.reactions.get_by_id(r).bounds = (0,0)

complete_result = phenotypic_phase_plane(gdls_kos_model,
                                         variables=[gdls_kos_model.reactions.get_by_id(target_biomass)],
                                         objective=gdls_kos_model.reactions.get_by_id(target_reaction),
                                         points=20)


In [93]:
min_r_df = min_result.data_frame
int_r_df = int_result.data_frame
complete_r_df = complete_result.data_frame

#parameters derived from wt dataframe
x = r_df.iloc[:, 0].tolist()
y1 = r_df.iloc[:, 1].tolist()
y2 = r_df.iloc[:, 2].tolist()

#parameters derived from min design dataframe
min_x = min_r_df.iloc[:, 0].tolist()
min_y1 = min_r_df.iloc[:, 1].tolist()
min_y2 = min_r_df.iloc[:, 2].tolist()

#parameters derived from intermediate design dataframe
int_x = int_r_df.iloc[:, 0].tolist()
int_y1 = int_r_df.iloc[:, 1].tolist()
int_y2 = int_r_df.iloc[:, 2].tolist()

#parameters derived from complete design dataframe
complete_x = complete_r_df.iloc[:, 0].tolist()
complete_y1 = complete_r_df.iloc[:, 1].tolist()
complete_y2 = complete_r_df.iloc[:, 2].tolist()

#user needed input:
title = "<b>Phenotypic Phase Planes of designs</b>"
xaxes_title = "<b>Growth (h<sup>-1</sup>)</b>"
yaxes_title =  "<b>PHA production (mMol/h)</b>"

fig = go.Figure()

#wt envelope
fig.add_trace(go.Scatter(
    x=x, 
    y=y1,
    fill=None,
    mode='lines',
    line_color='linen',
    showlegend=False
    ))
fig.add_trace(go.Scatter(
    x=x,
    y=y2,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', 
    line_color='linen',
    name='WT'))
#minimal envelope
fig.add_trace(go.Scatter(
    x=min_x, 
    y=min_y1,
    fill=None,
    mode='lines',
    line_color='orange',
    showlegend=False
    ))
fig.add_trace(go.Scatter(
    x=min_x,
    y=min_y2,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', 
    line_color='orange',
    name='Minimal Design'))
#intermediate envelope
fig.add_trace(go.Scatter(
    x=int_x, 
    y=int_y1,
    fill=None,
    mode='lines',
    line_color='red',
    showlegend=False
    ))
fig.add_trace(go.Scatter(
    x=int_x,
    y=int_y2,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', 
    line_color='red',
    name='Intermediate Design'))
#complete envelope
fig.add_trace(go.Scatter(
    x=complete_x, 
    y=complete_y1,
    fill=None,
    mode='lines',
    line_color='darkred',
    showlegend=False
    ))
fig.add_trace(go.Scatter(
    x=complete_x,
    y=complete_y2,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', 
    line_color='darkred',
    name='Complete Design'))

fig.update_layout(
    width = 860,
    height = 800,
    title = dict(text=title, font=dict(size=40), x=0.5, xanchor='center')
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.77,
    xanchor="right",
    x=1.0,
    font=dict(size=26)
))

fig.update_xaxes(title=dict(text=xaxes_title, font=dict(size=40)) ,
                 range=[0, 1.05*max(x)],
                 showline=True,
                 linewidth=3.0,
                 linecolor='black')

fig.update_yaxes(title=dict(text=yaxes_title, font=dict(size=40)),
                 range=[0, 1.05*max(y2)],
                 showline=True,
                 linewidth=3.0,
                 linecolor='black')

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis = dict(tickfont = dict(size=28)),
    xaxis = dict(tickfont = dict(size=28))
)
fig.write_image("envelopes.svg") #after post editing this will be image 3D
fig.show()